### Analizar alpha de la estrategia con otros activos del SP500

En este notebook se calculan los alphas anualizados de la estrategia con los activos del dataset del SP500. 

Se comparan los alphas con el alpha de la estrategia VIXSI con SPY


In [1]:
#Importamos las librerias necesarias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import yfinance as yf
import requests

%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')



### Se cargar la señal vixsi

In [2]:
datos_vixsi = pd.read_csv("../datos/VIXSI.csv", index_col=0)
datos_vixsi.index = pd.to_datetime(datos_vixsi.index)

### Se carga el dataset del sp500

In [3]:
data = pd.read_csv("../datos/sp500.csv", index_col=0)
data.index = pd.to_datetime(data.index)

### Se calcula la estrategia únicamente en corto

cuando vixsi es 1, el activo está fuera de mercado, cuando es -1, está en corto


In [4]:
data = data.dropna(axis=1, how="all")
data.index = pd.to_datetime(data.index)
datos_vixsi = datos_vixsi.loc[data.index]

estrcorto = pd.DataFrame(
             np.array(np.log(data).diff()) * np.array((datos_vixsi.VIXSI.shift(1) - 1)/2).reshape(-1,1),
            index = data.index)
estrcorto.columns = data.columns

### Se calcula el alpha anual (año por año) de la estrategia aplicada a los activos

A partir de los retornos de la estrategia solo en corto, se calcula el alpha anualizando los retornos por año y multiplicándolos por 2

In [5]:
df_alpha_ann = estrcorto.resample('A').sum()*2
df_alpha_ann.index = df_alpha_ann.index.strftime('%Y')

In [6]:
alfasup = df_alpha_ann.columns[(df_alpha_ann.mean() > df_alpha_ann["SPY"].mean())]
print (f"% alpha anual activos que superan al alpha de SPY: {len(alfasup)/len(df_alpha_ann.columns)*100:.2f}%")
print (f"% alpha promedio anual de todos los alphas que superan al SPY: {df_alpha_ann[alfasup].mean().mean()*100:.2f}%")
print (f"% alpha anual promedio del SPY: {df_alpha_ann.SPY.mean()*100:.2f}%")


% alpha anual activos que superan al alpha de SPY: 50.97%
% alpha anual promedio anual de todos los alphas que superan al SPY: 14.64%
% alpha anual promedio del SPY: 10.06%


### Se guardan los alphas anuales de todos los activos para el siguiente notebook

In [7]:
df_alpha_ann.to_csv ("../datos/alphas_ann.csv")

### Resumen

La mitad de las acciones del SP500 tiene un alpha anual medio superior al SPY, y el alpha anual de los activos que superan a SPY es 6% más alto que el alpha del SPY, con lo que en principio no parace muy difícil encontrar una cartera de acciones del SP500 cuyo alpha anual supere al alpha del SPY para sustituirlo por dicha cartera en la cobertura de la estrategia VIXSI. En los siguientes notebooks se describirán técnicas de selección de carteras.
